In [ ]:
import os
import datetime
import uuid
import keras
from keras import backend
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import np_utils
from sklearn import model_selection, metrics
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

# Utility

In [ ]:
def unique_filename(type='uuid'):
    if type == 'datetime':
        filename = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
    else:
        filename = str(uuid.uuid4())
    return filename

def makenewfold(prefix='output_', type='datetime'):
    suffix = unique_filename('datetime')
    foldname = 'output_' + suffix
    os.makedirs(foldname)
    return foldname

def save_history_history(fname, history_history, fold=''):
    np.save(os.path.join(fold, fname), history_history)

def load_history_history(fname, fold=''):
    history_history = np.load(os.path.join(fold, fname)).item(0)
    return history_history

# 분산 방식 Modeling - OOP

In [ ]:
class CNN_Model_OOP(Model):
    def __init__(self, num_classes):
        super(CNN_Model_OOP, self).__init__()
        self.num_classes = num_classes

        self.conv2D_1 = Conv2D(32, kernel_size=(3, 3), activation='relu')
        self.conv2D_2 = Conv2D(64, kernel_size=(3, 3), activation='relu')
        self.maxPooling2D = MaxPooling2D(pool_size=(2, 2))
        self.dropout_1 = Dropout(0.25)
        self.dropout_2 = Dropout(0.5)
        self.flatten = Flatten()
        self.dense_1 = Dense(128, activation='relu')
        self.dense_2 = Dense(num_classes, activation='softmax', name='pred')

    def call(self, x):
        h = self.conv2D_1(x)
        h = self.conv2D_2(h)
        h = self.maxPooling2D(h)
        h = self.dropout_1(h)
        h = self.flatten(h)

        h = self.dense_1(h)
        h = self.dropout_2(h)

        y = self.dense_2(h)

        return y

# Data - CIFAR10

In [ ]:
class Data_CIFAR10():
    def __init__(self, x, y, num_classes, scaling=True, test_size=0.2, random_state=0):
        self.x = x
        self.add_channels()
        x = self.x

        x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2, random_state=random_state)
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')

        if scaling:
            scaler = MinMaxScaler()
            n = x_train.shape[0]
            x_train = scaler.fit_transform(x_train.reshape(n, -1)).reshape(x_train.shape)
            n = x_test.shape[0]
            x_test = scaler.transform(x_test.reshape(n, -1)).reshape(x_test.shape)
            self.scaler = scaler

        y_train = np_utils.to_categorical(y_train, num_classes)
        y_test = np_utils.to_categorical(y_test, num_classes)
        
        self.x_train, self.x_test = x_train, x_test
        self.y_train, self.y_test = y_train, y_test

    def add_channels(self):
        x = self.x

        if len(x.shape) == 3:
            n, img_rows, img_cols = x.shape

            if backend.image_dim_ordering() == 'th':
                x = x.reshape(x.shape[0], 1, img_rows, img_cols)
                input_shape = (1, img_rows, img_cols)
            else:
                x = x.reshape(x.shape[0], img_rows, img_cols, 1)
                input_shape = (img_rows, img_cols, 1)
        else:
            input_shape = x.shape[1:]

        self.x = x
        self.input_shape = input_shape

# Training and Evaluation

In [ ]:
class Machine():
    def __init__(self, x, y, num_classes=2, fig=True):
        self.num_classes = num_classes
        self.set_data(x, y)
        self.set_model()
        self.fig = fig

    def set_data(self, x, y):
        self.data = Data_CIFAR10(x, y, self.num_classes)

    def set_model(self):
        self.model = CNN_Model_OOP(num_classes=self.num_classes)
        self.model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    def fit(self, epochs=10, batch_size=128, verbose=1):
        history = self.model.fit(self.data.x_train, self.data.y_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(self.data.x_test, self.data.y_test))
        return history

    def run(self, epochs=100, batch_size=128, verbose=1):
        history = self.fit(epochs=epochs, batch_size=batch_size, verbose=verbose)
        score = self.model.evaluate(self.data.x_test, self.data.y_test, verbose=0)

        y_test_pred = self.model.predict(self.data.x_test, verbose=0)
        y_test_pred = np.argmax(y_test_pred, axis=1)
        print(metrics.confusion_matrix(self.data.y_test, y_test_pred))
        print(f'Test score: {score[0]}')
        print(f'Test accuracy: {score[1]}')

        suffix = unique_filename('datetime')
        foldname = 'output_' + suffix
        os.makedirs(foldname)
        save_history_history('history.npy', history.history, fold=foldname)
        self.model.save_weights(os.path.join(foldname, 'lenet_model.h5'))

        if self.fig:
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)
            self.plot_acc(history)
            plt.subplot(1, 2, 2)
            self.plot_loss(history)
            plt.show()

    def plot_loss(self, history, title=None):
        if not isinstance(history, dict):
            history = history.history

        plt.plot(history['loss'])
        plt.plot(history['val_loss'])
        if title is not None:
            plt.title(title)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(['Training', 'Validation'], loc=0)

    def plot_acc(self, history, title=None):
        if not isinstance(history, dict):
            history = history.history

        plt.plot(history['accuracy'])
        plt.plot(history['val_accuracy'])
        if title is not None:
            plt.title(title)
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(['Training', 'Validation'], loc=0)

# Usage

In [ ]:
class LeNetMachine(Machine):
    def __init__(self):
        (x, y), (x_test, y_test) = cifar10.load_data()
        super().__init__(x, y, num_classes=10)

le_net = LeNetMachine()
le_net.run()